<center><img src='./imagenes/exploracion.png'></center>

In [25]:
import os
from zipfile import ZipFile
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from Code.utils import draw_text, stitch_images, imagenes_muestreo, get_size_statistics, corrplot

import numpy as np

plt.rcParams.update({'font.size': 15})
# plt.style.use('seaborn-white')

In [26]:
with ZipFile('database/smear2005.zip') as comprimido:
     comprimido.extractall('database')

In [44]:
# Leemos el archivo y lo cargamos en un DataFrame del modulo Pandas
datos = pd.read_excel('Code/database/smear2005/new_database_results.xls')

In [0]:
# Analizamos los primeros elementos
datos.head()

In [0]:
# Ahora veremos que columnas tiene el DataFrame
datos.columns

In [0]:
# La columna "Class" contiene la clase en numero a la que pertenece cada imagen
# Para un mejor entendimiento, vamos a convertir esos numeros en clases categoricas
# Para ello vamos a crear tres diccionarios
# El primero va a tomar los numeros del 0 al 6 que representan las 7 clases y los convertira a categoricos
# Los categoricos salen de los nombres de la carpeta donde se encuentran las imagenes para el caso de 7 clases
# Para el caso de 2 clases, vamos a requerir dos diccionarios
# El primero convierte la clase categorica a 0 o 1 dependiendo si es anormal o anormal
# El segundo nos va a convertir esos 0 y 1 en su clase categorica correspondiente
map_7_classes = {
    0: 'normal_superficiel',
    1: 'normal_intermediate',
    2: 'normal_columnar',
    3: 'light_dysplastic',
    4: 'moderate_dysplastic',
    5: 'severe_dysplastic',
    6: 'carcinoma_in_situ'
}

map_2_classes = {
    0: 'normal', 
    1: 'anormal'
}

map_normal_anormal = {
    'normal_superficiel': 0,
    'normal_intermediate': 0,
    'normal_columnar': 0,
    'light_dysplastic': 1,
    'moderate_dysplastic': 1,
    'severe_dysplastic': 1,
    'carcinoma_in_situ': 1
}


In [0]:
# Procedemos a contar cuantas imagenes por clase se encuentran en la base de datos
datos.groupby('Class').count()

In [0]:
datos_correlacion = datos.drop(['ID', 'Class'], 1)
corr = datos_correlacion.corr()
plt.figure(figsize=(16,16))
corrplot(corr, 'database/correlacion')

In [0]:
# Las clases deben de comenzar en 0, pero en este caso comienzan en 1, por lo
# que restamos 1 a cada clase y creamos una nueva columna de clase numerica para el caso 7
datos['Class_num_7'] = datos['Class'] - 1

In [0]:
# Mapeamos el diccionario de clases para traducir de numero a clase 
# categorica y asi obtener el folder correcto
datos['Class_cat_7'] = datos['Class_num_7'].map(map_7_classes)

In [0]:
# Tomando en cuenta las dos clases, procedemos a mapear para obtener clases 
# binarias numericas
datos['Class_num_2'] = datos['Class_cat_7'].map(map_normal_anormal)

In [0]:
# Al final mapeamos para obtener dos clases categoricas: normal y anormal
datos['Class_cat_2'] = datos['Class_num_2'].map(map_2_classes)

In [0]:
# Contamos cuantos elementos hay para el caso de 2 clases
datos.groupby('Class_cat_2').count()

In [0]:
get_size_statistics('database/smear2005/New database pictures')

In [0]:
# Creamos la columnas que tiene la ruta absoluta de las imagenes
# Vamos a requerir la ruta absoluta en este caso
ruta = os.path.abspath('./database/smear2005/New database pictures/')
ruta
r = lambda x: os.path.join(ruta, os.path.join(x['Class_cat_7'],  x['ID']))
datos['file'] = datos.apply(r, axis=1)

In [0]:
# Ahora colocamos la columna con la ruta absoluta de las mascaras
datos['masks'] = datos['ID'].str.slice_replace(start=23, repl='-d.bmp')
rr = lambda x: os.path.join(ruta, os.path.join(x['Class_cat_7'],  x['masks']))
datos['file_masks'] = datos.apply(rr, axis=1)

In [0]:
#Guardamos el dataframe como .csv
datos.to_csv('database/database_explore.csv')

In [0]:
# Procedemos a tomar muestras de las imagenes para poder visualizar los tipos de celulas
n_muestras = 7
df_muestras = datos.groupby('Class_cat_7').apply(lambda x: x.sample(n=n_muestras))
# display(df_muestras)
ims = []

for file, y, in zip(df_muestras['file'], df_muestras['Class_cat_7']):
    img = cv2.imread(file)
    img = cv2.resize(img,(256,256))
    img = draw_text(img, f'{y}', color=(0, 255, 0), font_size=30,position=(10, 0), font=f'./fonts/computer-modern/cmunssdc.ttf') 
    ims.append(img)
    
stitched = stitch_images(ims, cols=7)    
imagenes_muestreo(stitched, nom_archivo='database/muestras_celulas')

In [0]:
n_muestras = 7
df_muestras = datos.groupby('Class_cat_7').apply(lambda x: x.sample(n=n_muestras))
# display(df_muestras)
ims = []

for file, y, in zip(df_muestras['file_masks'], df_muestras['Class_cat_7']):
    img = cv2.imread(file)
    img = cv2.resize(img,(256,256))
    img = draw_text(img, f'{y}', color=(0, 255, 0), font_size=30,position=(10, 0), font=f'./fonts/computer-modern/cmunssdc.ttf') 
    ims.append(img)
    
stitched = stitch_images(ims, cols=7)    
imagenes_muestreo(stitched, nom_archivo='database/muestras_mascaras')